<a href="https://colab.research.google.com/github/charava/Reddit/blob/main/RedditScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q praw
import praw
import numpy as np
import pandas as pd
from os.path import exists
#import cron...automate a weekly schedule for scraping
#octaparse, amazon lambda
#running it only takes 39 seconds

#https://towardsdatascience.com/scraping-reddit-data-1c0af3040768
reddit = praw.Reddit(client_id='izBlAjZ7nSAT62DPxU2thw', client_secret='9OwPwh_axISvTOZwWZN9g66im-C-AA', user_agent='Scraper2')

#reddit = praw.Reddit(client_id='9FOPiLyp8H5GnfCWa2O2ZA', client_secret='FlwV13e7uYP_O778lSZftNDJ0pCqqw', user_agent='WebScraping')
subreddit_names = ['inventionideas', 'appideas', 'annoyances', 'inventions', 'problems', 'lightbulb']
#maybe dont have lightbulb idk


#just delete the phrase from the row
unnecessary_phrases = ['hi','hello','thanks','thx',
'ty','tx','dear community','i am looking for',':)']

#this is a sign to delete the entire post/row
personal_phrases = ['my idea','my app','my invention','i\'ve got an idea',
'i\'ve got an app','i\'ve got an invention','i\'ve been thinking',
'i\'ve got this','my product','need your','feedback',]
# MY IDEA SEEMS TO GO BOTH WAYS...SOME CASES ACTUALLY ARENT PERSONAL


problem_keywords = ['problem', 'issue','bad','difficult','can\'t','trouble','hard','not good']
#need to identify people asking a # QUESTION:

idea_keywords = ['idea','solution','solve','improve','better','improve','fix','what if','propos',]
#could aslo sort from actual subreddit like all the app ideas and invention ideas
#AppIdeas
#inventionideas

ask_keywords = ['?','can you','need','ask','want','please']
not_ask_keywords = [] #if this is in an ask phrase, then it cant be an ask phrase and must be deleted




#number of loops for the different functions
FIRSTSCRAPEMAXPOSTS = 10
LOOPMAXPOSTS = 100
MAINFILE = 'first.csv'



#first scrape -- creating the first ever file
def firstScrape():
    for name in subreddit_names: #only want to do this once
        filename = name + '.csv'
        subreddit = reddit.subreddit(name)
        #sorting into individual csv files based on subreddit
        postList = []
        for post in subreddit.new(limit=FIRSTSCRAPEMAXPOSTS):
            postList.append([post.title, post.score, post.selftext, post.id])
        df = pd.DataFrame(postList, columns=['title','score','body','id'])

        if exists(filename):
            no_index(filename)
        else:
            df.to_csv(filename,mode='w',header=True, index=False)     #creating individual csv
            no_index(filename)

        if exists(MAINFILE):
            no_index(MAINFILE)
        else:
            df.to_csv(MAINFILE,mode='w',header=True,index=False)     #creating an amalgamated csv file
            no_index(MAINFILE)


#function to scrape once file has already been created
def loopScrape():
    for name in subreddit_names:
        filename = name + '.csv'
        subreddit = reddit.subreddit(name)
        #sorting into individual csv files based on subreddit
        postList = []
        for post in subreddit.new(limit=LOOPMAXPOSTS):
            #df = pd.read_csv(filename)
            #if post.title not in df['title']:
            postList.append([post.title, post.score, post.selftext, post.id])

        df = pd.DataFrame(postList, columns=['title','score','body','id'])
        df.to_csv(filename,mode='a', header=False, index=False)  #creating individual csv
        df.to_csv(MAINFILE,mode='a', header=False, index=False) #creating an amalgamated csv file


def no_dupes(filename):
    df = pd.read_csv(filename)
    df.drop_duplicates(subset=['title'],keep='first',inplace=True)
    df.to_csv(filename, index=False)

#deleting index columns to prevent index=False from being true somehow
def no_index(filename):
    df = pd.read_csv(filename)
    column_headers = df.columns.values
    for number in range(len(column_headers)):
        name = column_headers[number]
        loweredname = name.lower()
        if 'unnamed' in loweredname:
            df.drop(labels=name, axis=1,inplace=True) #used to be df =   and had inplace False
    df.to_csv(filename, index=False)


# copy dfcheck2 and then when personal prhase found in dfcheck2, modify dfcheck3
#cleaning for the first.csv file
def cleanData(filename):
    dfcheck2 = pd.read_csv(filename)
    no_dupes(filename)
    no_index(filename)


    #replacing special characters
    dfcheck2=dfcheck2.replace('\*','',regex=True)
    dfcheck2=dfcheck2.replace('&#x200B;','',regex=True) #RIGHT NOW NOT DELETING


    # clean bad words

    #taking out unnecesary phrases



    #once replacing special characters has happened
    dfcheck3 = dfcheck2.copy(deep=True)
    cleaned_file = 'clean' + filename


    dropped_stuff = []

    #deleting rows with personal phrases in them
    for row in range(len(dfcheck2)):
        row_title = dfcheck2.loc[row, 'title']
        row_body = dfcheck2.loc[row,'body']

        for phrase in personal_phrases:
            phrase = phrase.lower()

            if isinstance(row_title, str):
                row_title = row_title.lower()
                if phrase in row_title:
                    #print('found in row title')
                    #print('filename = ' + filename + '  row = ' + str(row))
                    dfcheck3.drop(labels=row,axis=0, inplace=True)
                    dropped_stuff.append(dfcheck2.loc[row]) #seeing what is NOT on newfirst
                    break

                elif isinstance(row_body, str):
                    row_body = row_body.lower()
                    if phrase in row_body:
                        #print('found in body')
                        #print('filename = ' + filename + '  row = ' + str(row))
                        dfcheck3.drop(labels=row,axis=0,inplace=True)
                        dropped_stuff.append(dfcheck2.loc[row]) #seeing what is NOT on newfirst
                        break


    dfcheck3.to_csv(cleaned_file,mode='w',index=False)
    no_index(cleaned_file)
    no_dupes(cleaned_file)


    df = pd.DataFrame(dropped_stuff, columns=['title','score','body','id'])
    if exists('droppedstuff.csv'):
        df.to_csv('droppedstuff.csv',mode='a', header=False, index=False)
    else:
        df.to_csv('droppedstuff.csv',mode='w', header=True, index=False)
    no_dupes('droppedstuff.csv')
    no_index('droppedstuff.csv')


    no_index(filename)
    no_dupes(filename)




def findProblems(filename): #eh take out...too ambiguous to just look for key words
    check = pd.read_csv(filename)
    no_dupes(filename)
    no_index(filename)

    unsorted = []
    identifiedList = []
    for row in range(len(check)):
        row_title = check.loc[row, 'title']
        row_body = check.loc[row,'body']

        for phrase in problem_keywords:
            phrase = phrase.lower()

            if isinstance(row_title, str):
                row_title = row_title.lower()
                if phrase in row_title:
                    identifiedList.append(check.loc[row])
                    break

                elif isinstance(row_body, str):
                    row_body = row_body.lower()
                    if phrase in row_body:
                        identifiedList.append(check.loc[row])
                        break
                    else:
                        unsorted.append(check.loc[row])
                        break

                else:
                    unsorted.append(check.loc[row])



    df = pd.DataFrame(identifiedList, columns=['title','score','body','id'])
    if exists('identifiedproblems.csv'):
        df.to_csv('identifiedproblems.csv',mode='a', header=False, index=False)
    else:
        df.to_csv('identifiedproblems.csv',mode='w', header=True, index=False)  #to get a header initially
    no_dupes('identifiedproblems.csv')
    no_index('identifiedproblems.csv')


    anotherdf = pd.DataFrame(identifiedList, columns=['title','score','body','id'])
    if exists('unsorted.csv'):
        anotherdf.to_csv('unsorted.csv',mode='a', header=False, index=False)
    else:
        anotherdf.to_csv('unsorted.csv',mode='w', header=True, index=False)  #to get a header initially
    no_dupes('unsorted.csv')
    no_index('unsorted.csv')



def findIdeas(filename): #eh take out...too ambiguous to just look for key words
    check = pd.read_csv(filename)
    no_dupes(filename)
    no_index(filename)


    identifiedList = []
    for row in range(len(check)):
        row_title = check.loc[row, 'title']
        row_body = check.loc[row,'body']

        for phrase in idea_keywords:
            phrase = phrase.lower()

            if isinstance(row_title, str):
                row_title = row_title.lower()
                if phrase in row_title:
                    identifiedList.append(check.loc[row])
                    break

                elif isinstance(row_body, str):
                    row_body = row_body.lower()
                    if phrase in row_body:
                        identifiedList.append(check.loc[row])
                        break


    df = pd.DataFrame(identifiedList, columns=['title','score','body','id'])
    if exists('identifiedideas.csv'):
        df.to_csv('identifiedideas.csv',mode='a', header=False, index=False)
    else:
        df.to_csv('identifiedideas.csv',mode='w', header=True, index=False)  #to get a header initially
    no_dupes('identifiedideas.csv')
    no_index('identifiedideas.csv')



def findAsks(filename): #eh take out...too ambiguous to just look for key words
    check = pd.read_csv(filename)
    no_dupes(filename)
    no_index(filename)


    identifiedList = []
    for row in range(len(check)):
        row_title = check.loc[row, 'title']
        row_body = check.loc[row,'body']

        for phrase in ask_keywords:
            phrase = phrase.lower()

            if isinstance(row_title, str):
                row_title = row_title.lower()
                if phrase in row_title:
                    identifiedList.append(check.loc[row])
                    break

                elif isinstance(row_body, str):
                    row_body = row_body.lower()
                    if phrase in row_body:
                        identifiedList.append(check.loc[row])
                        break


    df = pd.DataFrame(identifiedList, columns=['title','score','body','id'])
    if exists('identifiedasks.csv'):
        df.to_csv('identifiedasks.csv',mode='a', header=False, index=False)
    else:
        df.to_csv('identifiedasks.csv',mode='w', header=True, index=False)  #to get a header initially
    no_dupes('identifiedasks.csv')
    no_index('identifiedasks.csv')



def overlapped():   #overlapped is consolidating all the posts that fall under all three categories: problem, idea, and ask
    problems = pd.read_csv('identifiedproblems.csv')
    ideas = pd.read_csv('identifiedideas.csv')
    asks = pd.read_csv('identifiedasks.csv')

    identifiedList = []

    for problems_row in range(len(problems)):
        problems_post = problems.loc[problems_row, 'title']

        for ideas_row in range(len(ideas)):
            ideas_post = ideas.loc[ideas_row, 'title']

            for asks_row in range(len(ideas)):
                asks_post  = asks.loc[asks_row, 'title']

                if problems_post == ideas_post:
                    identifiedList.append(ideas.loc[ideas_row])
                    break
                elif asks_post == problems_post:
                    identifiedList.append(ideas.loc[problems_row])
                    break
                elif asks_post == ideas_post:
                    identifiedList.append(ideas.loc[asks_row])
                    break

    #not efficient code, copies thousands of repeats to csv then deletes dupes at the very end

    df = pd.DataFrame(identifiedList, columns=['title','score','body','id'])
    if exists('overlapped.csv'):
        df.to_csv('overlapped.csv',mode='a', header=False, index=False)
    else:
        df.to_csv('overlapped.csv',mode='w', header=True, index=False)  #to get a header initially

    no_dupes('overlapped.csv')
    no_index('overlapped.csv')



#-------------------------

# THE ACTUAL SCRAPING
firstScrape()
loopScrape()

no_dupes(MAINFILE)
no_index(MAINFILE)

for name in subreddit_names:
    filename = name + '.csv'
    no_dupes(filename)
    no_index(filename)


#CLEANING DATA
for name in subreddit_names:
    filename = name + '.csv'
    cleanData(filename)

cleanData(MAINFILE)

findProblems(MAINFILE) #eh take out
findIdeas(MAINFILE)
findAsks(MAINFILE)
overlapped()


#returns identifiedasks, identifiedproblems, identifiedideas



#add more personal personal_phrases
#delete unneded phrases, inplace=True
#make list of innapropro words and replace them
#make sure its replacing unnecessary phrases cuz its no, adde emojis to unnecessary phrase list
#AND clean stop words, but append to new file, perhaps combine this step with dfcheck3
#fix spelling errors

#Clean inappropriate, swear words (github cleaner code)
# https://github.com/vzhou842/profanity-check/tree/master/profanity_check
#replaced "fucked" "messed"

#deleting stuff around relationships?

#sorting into categories using text classification like ckimate change, health for
#displaying on the website



'''
QUESTIONS:
Why does it take so long to scrape in my program? Is this literally just the scraping
or is my code just not efficient?

scraping on a schedule? run on a server and import datetime to scrape every week
can i have my code run and scrape every week

should i eventually just drop the indiviudal files

THOUGHTS:
for the actual front end website, create a feature for the mod
to delete posts easily by clicking an x on the interface


CSV KEY:
dropped stuff = stuff that is not cleandata() thought had a personal phrase and shouldnt
be accounted for

first, inventionideas, lightbulb...etc = the original scrape straight from reddit

newannoyances, newfirst...etc = went through cleandata() and is everything that didn't
have personal phrases

identifiedasks = sorted "asks" or need statements from the new___ files
identifiedproblems = sorted "problem" statements from new____ files
identifiedideas = sorted "idea" statements from new___ files

overlapped = all the posts that got sorted into 2 or more categories: ask and a problem, idea and a problem, problem and ask

stopwords = so far not used, needed though if want to strip down strings to key words ONLY

unsorted = stuff from new___ that didn't qualify as an ask, idea, or problem based on find___ functions
'''

     |████████████████████████████████| 176 kB 4.2 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readt

'\nQUESTIONS:\nWhy does it take so long to scrape in my program? Is this literally just the scraping\nor is my code just not efficient?\n\nscraping on a schedule? run on a server and import datetime to scrape every week\ncan i have my code run and scrape every week\n\nshould i eventually just drop the indiviudal files\n\nTHOUGHTS:\nfor the actual front end website, create a feature for the mod\nto delete posts easily by clicking an x on the interface\n\n\nCSV KEY:\ndropped stuff = stuff that is not cleandata() thought had a personal phrase and shouldnt\nbe accounted for\n\nfirst, inventionideas, lightbulb...etc = the original scrape straight from reddit\n\nnewannoyances, newfirst...etc = went through cleandata() and is everything that didn\'t\nhave personal phrases\n\nidentifiedasks = sorted "asks" or need statements from the new___ files\nidentifiedproblems = sorted "problem" statements from new____ files\nidentifiedideas = sorted "idea" statements from new___ files\n\noverlapped = al

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers numpy torch sklearn datasets


In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)


training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")


trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)
trainer.train()


metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()


ModuleNotFoundError: ignored

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("imdb")
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
inputs = tokenizer(sentences, padding="max_length", truncation=True)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]


model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)


training_args = TrainingArguments("test_trainer")



trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)
trainer.train()



metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.evaluate()




training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")



model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

tf_train_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)


model.save_pretrained("my_imdb_model")
pytorch_model = AutoModelForSequenceClassification.from_pretrained("my_imdb_model", from_tf=True)
'''

del model
del pytorch_model
del trainer
torch.cuda.empty_cache()'''

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

ModuleNotFoundError: ignored

In [ ]:
#SECOND TRY WITH TINY BERT DIDNT WORK
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
#from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split


def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)


# check text classification models here: https://huggingface.co/models?filter=text-classification
model_name = "prajjwal1/bert-tiny"
#model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
#tokenizer = AutoModel.from_pretrained(model_name)

#model_name = AutoModel.from_pretrained("prajjwal1/bert-tiny")



def read_20newsgroups(test_size=0.2):
  # download & load 20newsgroups dataset from sklearn's repos
  dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
  documents = dataset.data
  labels = dataset.target
  # split into training & testing a return data as well as label names
  return train_test_split(documents, labels, test_size=test_size), dataset.target_names

# call the function
(train_texts, valid_texts, train_labels, valid_labels), target_names = read_20newsgroups()


# tokenize the dataset, truncate when passed `max_length`,
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)

# load the model and pass to CUDA
#model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")
model = AutoModelForSequenceClassification.from_pretrained(model_name)
#model = AutoModel.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")

from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)


trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)


# train the model
trainer.train()

# evaluate the current model after training
trainer.evaluate()


# saving the fine tuned model & tokenizer
model_path = "20newsgroups-bert-base-uncased"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

RuntimeError: ignored

In [ ]:
#UNTOUCHED OG MODEL
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)


# check text classification models here: https://huggingface.co/models?filter=text-classification
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)


def read_20newsgroups(test_size=0.2):
  # download & load 20newsgroups dataset from sklearn's repos
  dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
  documents = dataset.data
  labels = dataset.target
  # split into training & testing a return data as well as label names
  return train_test_split(documents, labels, test_size=test_size), dataset.target_names

# call the function
(train_texts, valid_texts, train_labels, valid_labels), target_names = read_20newsgroups()


# tokenize the dataset, truncate when passed `max_length`,
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)

# load the model and pass to CUDA
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")


from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)


trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)


# train the model
trainer.train()

# evaluate the current model after training
trainer.evaluate()


# saving the fine tuned model & tokenizer
model_path = "20newsgroups-bert-base-uncased"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]

text = """
hello im scard of dogs

"""
print(get_prediction(text))

sci.electronics
